In [1]:
import albumentations
import albumentations.pytorch
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tqdm import tqdm

import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
from sklearn.metrics import f1_score
from efficientnet_pytorch import EfficientNet

tqdm.pandas()

/opt/conda/lib/python3.8/site-packages/tqdm/std.py:703: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"{device} is using!")

cuda:0 is using!


In [7]:
NUM_EPOCHS = 15
BATCH_SIZE = 4
OUTPUT_CLASS = 18
LEARNING_RATE = 1e-4

In [8]:
class MyDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']  # transforms를 사용하시는 분은 X = self.transform(X)
        return X, y

In [9]:
class TestDataset(Dataset):
    def __init__(self, path, label, transform):
        img_list = []
        for p in tqdm(path):
            img = cv2.imread(p)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append(img)
        
        self.X = img_list
        self.y = label
        self.transform = transform

    def __len__(self):
        len_dataset = len(self.X)
        return len_dataset

    def __getitem__(self, idx):
        X,y = self.X[idx], self.y[idx]
        X = self.transform(image=X)['image']  # transforms를 사용하시는 분은 X = self.transform(X)
        return X

In [10]:
# Augmentation은 각자 방식대로

train_transform = albumentations.Compose(
  [
      albumentations.Resize(600,600),
#       albumentations.RandomRotation(15),
#       albumentations.HorizontalFlip(p=0.3),
      albumentations.OneOf([albumentations.ShiftScaleRotate(rotate_limit=15, p=0.5),
                            albumentations.MotionBlur(p=0.5),
                            albumentations.OpticalDistortion(p=0.5),
                            albumentations.GaussNoise(p=0.5)], p=1),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2(),
      #       이미지 원본 사이즈는 384, 512   
  ]
)

test_transform = albumentations.Compose(
  [
      albumentations.Resize(600,600),
      albumentations.Normalize((0.548, 0.504, 0.479), (0.237, 0.247, 0.246)),
      albumentations.pytorch.transforms.ToTensorV2()
      #       이미지 원본 사이즈는 384, 512   
  ]
)

In [11]:
# DF 읽어오기
train_df = pd.read_csv('/opt/ml/input/train_df.csv')
valid_df = pd.read_csv('/opt/ml/input/valid_df.csv')

# Dataset 객체 생성
dataset_train = MyDataset(path=train_df['full_path'].values,
                          label=train_df['label'].values,
                          transform=train_transform)

dataset_valid = MyDataset(path=valid_df['full_path'].values,
                          label=valid_df['label'].values,
                          transform=test_transform)

# Loader 올리기
train_dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
valid_dataloader = DataLoader(dataset_valid, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

100%|██████████| 3780/3780 [00:10<00:00, 347.47it/s]
